# Microsoft Recommendations

In [1]:
# %run code/add_data.py
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,}'.format
from pandasql import sqldf
import json
import requests
import os
import seaborn as sns


import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
#os.system("ls data/zippedData | grep sv > file_list.txt")




In [6]:
%run code/data_preparations.py
csv_dict.keys()

In [4]:
full_profit_df.head(2)

,id,tconst,title,release_date,revenue,budget,net_profit,year,vote_average,popularity,vote_count,runtime,genres
0,5,tt0113101,Four Rooms,1995-12-09,"4,257,354.0","4,000,000.0","257,354.0",1995,5,15,2123,98.0,"[80, 35]"
1,11,tt0076759,Star Wars,1977-05-25,"775,398,007.0","11,000,000.0","764,398,007.0",1977,8,90,16953,121.0,"[12, 28, 878]"


In [7]:
full_profit_df["genre_id"]=full_profit_df['genres'].apply(clean_genres)
full_profit_df.drop(columns='genres', inplace=True)
full_profit_by_genre_id=full_profit_df.explode('genre_id')

full_profit_by_genre=full_profit_by_genre_id.merge()

In [8]:
csv_dict.keys()

dict_keys(['bom.movie_gross', 'imdb.name.basics', 'imdb.title.akas', 'imdb.title.basics', 'imdb.title.crew', 'imdb.title.principals', 'imdb.title.ratings', 'rt.movie_info', 'rt.reviews', 'tmdb.movies', 'tmdb_genres', 'tmdb_imdb_gross', 'tmdb_imdb_gross_full', 'tmdb_movie_ids', 'tn.movie_budgets'])

In [ ]:
summary_df=sep_genre_df.groupby(['genre'], as_index=False).mean().sort_values(by="total_gross", ascending=False)[['genre','total_gross']]
#summary_df=summary_df.loc[:22]
summary_df